## Scrapper de seloger.com

In [1]:

import json
import time


import requests

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options



import random

from selenium.webdriver.common.keys import Keys

import time

import re

###  Chargement du Navigateur

In [2]:
options = webdriver.ChromeOptions()

options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument("window-size=1920,1080")
driver = webdriver.Chrome('/home/moulouel/drivers/chromedriver', options=options)


driver.delete_all_cookies()



In [3]:
driver.get("http://www.seloger.com")

## Choisir la location 

In [7]:
type_achat=driver.find_element_by_xpath("//label[contains(text(),'Louer')]")
type_achat.click()
time.sleep(random.randint(3,10))

### Barre de recherche villes pour mettre France

In [4]:
location_form=driver.find_element_by_xpath("//input[@id='agatha_autocomplete_autocompleteUI__input']")

location_form.send_keys("France")




In [5]:
time.sleep(5)
location_form.send_keys(Keys.RETURN);

## Simulation comportement humain avec temps de lecture pour cliquer sur rechercher

In [6]:
time.sleep(random.randint(3,10))

## Click sur le bouton de recherche

In [8]:
#Mettre sleep
search_bar=driver.find_element_by_xpath("//a[contains(text(),'Rechercher')]")

search_bar.click()

## On tombe sur la page des annonces

In [39]:
annonces=driver.find_elements_by_xpath("//div[@data-test='sl.card-container']")


## On ouvre toutes les annonces de seloger de la page dans de nouveau onglets

In [335]:
seloger_part=re.compile("www\.seloger\.")
    
for annonce in annonces[1:3]:   
    lien_annonce=annonce.find_element_by_xpath(".//a[@name='classified-link']")
    a=seloger_part.findall(lien_annonce.get_attribute("href"))
    if(len(a)>0):
        print(lien_annonce)
        time.sleep(random.randint(20,60))
        webdriver.ActionChains(driver).move_to_element(lien_annonce).perform()
        lien_annonce.click()    
    

<selenium.webdriver.remote.webelement.WebElement (session="3be8d18370d60e153b4f4880ac936080", element="6f1fcf84-7446-479e-85f8-4d0d70e05370")>


## On scrappe les informations

In [336]:
bien={}

In [343]:
for onglet in driver.window_handles[1:]:
    driver.switch_to_window(onglet)
    time.sleep(random.randint(0,10))
    #on recupere le numero te telephone
    affiche_num=driver.find_element_by_xpath("//button[contains(text(),'Affichez le numéro')]")
    webdriver.ActionChains(driver).move_to_element(affiche_num).perform()
    affiche_num.click()
    time.sleep(random.randint(0,10))
    telephone=driver.find_element_by_xpath("//a[contains(@href,'tel:')]").get_attribute("href")
    bien['telephone']=telephone.split(':')[1]
    #on recupere l'agence qui fournit le bien
    agence_block=driver.find_element_by_xpath("//div[@data-test='side-agency-block']")
    bien['agence']=agence_block.find_element_by_xpath(".//h3").text
    
    #on recupere le type de bien et le quartier et la ville
    infos_bien=driver.find_element_by_xpath("//div[contains(@class,'Summarystyled__Title')]")


    separators=re.compile('\n|,\s')

    type_bien=re.split(separators,infos_bien.text)
    bien['type_bien']=type_bien[0]

    #    bien['quartier']=data_location[1]

    #    bien['ville']=data_location[2]

    other_infos=infos_bien.find_elements_by_xpath(".//following-sibling::*")
    prix_div=driver.find_element_by_xpath("//div[contains(text(),'Le loyer mensuel est de')]")
    charges_div=prix_div.find_elements_by_xpath(".//following-sibling::*")[1]
    localisation_div=driver.find_elements_by_xpath("//p[contains(text(),'Le bien')]")



    piece=re.compile('\d+ pièce')
    chambre=re.compile('\d+ chambre')
    surface=re.compile('\d+\,?\d*m²')
    price=re.compile("\d+\,?\d*(?= €)")
    charges=re.compile("\d+\,?\d*(?= €)")
    code_postale_pattern=re.compile("\d{5}")
    quartier_pattern=re.compile("quartier.*")

    for info in other_infos:

        #print(i.text)
    #    print(i.text)
        #print(re.search(piece,i.text))

        if(piece.findall(info.text)):
            bien['pieces']=piece.findall(info.text)[0]
        if(chambre.findall(info.text)):
            bien['chambres']=chambre.findall(info.text)[0]
        if(surface.findall(info.text)):
            bien['surface']=surface.findall(info.text)[0]

        if(price.findall(prix_div.text)):
            bien['prix']=price.findall(prix_div.text)[0]
        if(charges.findall(prix_div.text)):
            bien['charges']=charges.findall(charges_div.text)[0]
        if(code_postale_pattern.findall(localisation_div[0].text)):
            bien['zip_code']=code_postale_pattern.findall(localisation_div[0].text)[0]
        if(quartier_pattern.findall(localisation_div[0].text)):
            bien['quartier']=quartier_pattern.findall(localisation_div[0].text)[0]
    reference_annonce=driver.find_element_by_xpath("//div[contains(text(),'Réf. annonce')]")
    bien['reference_annonce']=reference_annonce.text.split(':')[1].strip()



/home/moulouel/anaconda3/envs/BigDataEnv/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: use driver.switch_to.window instead
  


In [344]:
bien

{'telephone': '0496125172',
 'agence': 'AGENCE IMMOBILIERE DES TANNEURS',
 'type_bien': 'Appartement',
 'prix': '530',
 'charges': '30',
 'zip_code': '13005',
 'quartier': 'quartier Le Camas.',
 'pieces': '1 pièce',
 'surface': '21,5m²',
 'reference_annonce': 'L000435'}